# LCA without background

In [1]:
bd.projects.set_current('default')

NameError: name 'bd' is not defined

In [2]:
import os
import sys
sys.path.append(os.path.abspath(".."))
from bw_exiobase.background_importer import *
from bw_exiobase.foreground_importer import *
from bw_exiobase.datapackage_builder import *
from bw_exiobase.uncertainty_handler import *
from bw_exiobase.exiobase_lca import *
from constants import *

In [3]:
for i in bd.methods:
    if "IPCC" in i[1]:
        print(i)

('ecoinvent-3.11', 'IPCC 2013 no LT', 'climate change no LT', 'global temperature change potential (GTP100) no LT')
('ecoinvent-3.11', 'IPCC 2013 no LT', 'climate change no LT', 'global temperature change potential (GTP20) no LT')
('ecoinvent-3.11', 'IPCC 2013 no LT', 'climate change no LT', 'global warming potential (GWP100) no LT')
('ecoinvent-3.11', 'IPCC 2013 no LT', 'climate change no LT', 'global warming potential (GWP20) no LT')
('ecoinvent-3.11', 'IPCC 2013', 'climate change', 'global temperature change potential (GTP100)')
('ecoinvent-3.11', 'IPCC 2013', 'climate change', 'global temperature change potential (GTP20)')
('ecoinvent-3.11', 'IPCC 2013', 'climate change', 'global warming potential (GWP100)')
('ecoinvent-3.11', 'IPCC 2013', 'climate change', 'global warming potential (GWP20)')
('ecoinvent-3.11', 'IPCC 2021 (incl. biogenic CO2) no LT', 'climate change: biogenic (incl. CO2) no LT', 'global warming potential (GWP100) no LT')
('ecoinvent-3.11', 'IPCC 2021 (incl. biogeni

In [20]:
bd.databases

Databases dictionary with 2 object(s):
	ecoinvent-3.11-biosphere
	ecoinvent-3.11-consequential

In [32]:
#bd.Database('ecoinvent-3.11-biosphere').get(134929304525901883)
bioexc = bd.Database('ecoinvent-3.11-biosphere').random()

In [38]:
bioexc['code']

'c33d4b0b-99ac-5ec3-a825-d84a39f9f0eb'

In [40]:
bioexc.id

134929304915972145

In [56]:
#for node in bd.Database('ecoinvent-3.11-biosphere'):
#    print(node.id)


In [58]:
for node in bd.Database('ecoinvent-3.11-biosphere'):
    if node.id == 134929304915972145:
        print(node['code'])
    

c33d4b0b-99ac-5ec3-a825-d84a39f9f0eb


In [60]:
[node['code'] for i in bd.Database('ecoinvent-3.11-biosphere') if i.id == 134929304915972145]

['243cdfb9-053a-509a-9a34-423a77b10309']

In [46]:
bioexc = bd.Database('ecoinvent-3.11-biosphere').get('c33d4b0b-99ac-5ec3-a825-d84a39f9f0eb')

In [47]:
bioexc = bd.Database('ecoinvent-3.11-biosphere').get('134929304915972145')

UnknownObject: 

In [44]:
bioexc.as_dict()

{'categories': ('soil',),
 'code': 'c33d4b0b-99ac-5ec3-a825-d84a39f9f0eb',
 'CAS number': '688046-61-9',
 'synonyms': [],
 'name': 'Pyriofenone',
 'database': 'ecoinvent-3.11-biosphere',
 'unit': 'kilogram',
 'type': 'emission',
 'id': 134929304915972145}

In [61]:
ipcc = ('ecoinvent-3.11', 'IPCC 2013', 'climate change', 'global warming potential (GWP100)')
bw_method = bd.Method(ipcc).load()
bw_method
#method_df = pd.DataFrame(bw_method, columns=["database_code", "cf_number"])
#method_df


[(134929304525901883, 1301.27019996),
 (134929304538484768, 1301.27019996),
 (134929304878223428, 1301.27019996),
 (134929304819503223, 1301.27019996),
 (134929304915972213, 160.09863099),
 (134929304500736020, 160.09863099),
 (134929304509124680, 160.09863099),
 (134929304555262030, 160.09863099),
 (134929304446210172, 4804.43907777),
 (134929304924360730, 4804.43907777),
 (134929304702062679, 4804.43907777),
 (134929304387489829, 4804.43907777),
 (134929304555262025, 5823.72618701),
 (134929304890806279, 5823.72618701),
 (134929305008246858, 5823.72618701),
 (134929304588816407, 5823.72618701),
 (134929304643342398, 782.03725986),
 (134929304769171502, 782.03725986),
 (134929304890806318, 782.03725986),
 (134929304991469640, 782.03725986),
 (134929304462987389, 137.56020393),
 (134929304899194975, 137.56020393),
 (134929304853057653, 137.56020393),
 (134929304769171572, 137.56020393),
 (134929304471375957, 8592.20208449),
 (134929304404267077, 8592.20208449),
 (134929304802725923, 85

In [7]:
method_df["database_code"].to_list()

[134929304525901883,
 134929304538484768,
 134929304878223428,
 134929304819503223,
 134929304915972213,
 134929304500736020,
 134929304509124680,
 134929304555262030,
 134929304446210172,
 134929304924360730,
 134929304702062679,
 134929304387489829,
 134929304555262025,
 134929304890806279,
 134929305008246858,
 134929304588816407,
 134929304643342398,
 134929304769171502,
 134929304890806318,
 134929304991469640,
 134929304462987389,
 134929304899194975,
 134929304853057653,
 134929304769171572,
 134929304471375957,
 134929304404267077,
 134929304802725923,
 134929304618176541,
 134929304735617073,
 134929304588816412,
 134929304676896817,
 134929304853057657,
 134929304974692448,
 134929304915972129,
 134929304811114535,
 134929304412655644,
 134929304752394311,
 134929304861446215,
 134929304462987383,
 134929304710451209,
 134929304446210134,
 134929304651731005,
 134929304752394340,
 134929304387489882,
 134929304693674076,
 134929304379101286,
 134929304446210166,
 134929304462

In [6]:
#method_df[["database", "code"]] = method_df["database_code"].to_list()

cf_selected = method_df[method_df["database_code"].isin(codes)][["code", "cf_number"]]

ValueError: Columns must be same length as key

1. Get technosphere matrix

In [4]:
bg_importer = BackgroundImporter()

tech_df = pd.read_table(EXIOBASE_AGGREGATED_A_FILE, sep='\t', header=None, low_memory=False)
raw_tech = tech_df.iloc[3:, 2:].astype('float').to_numpy()
tech_matrix = bg_importer.form_tech_matrix(raw_tech)

2. Get biosphere matrix

In [5]:
bio_df = pd.read_csv(EXIOBASE_AGGREGATED_S_FILE, header=[0,1], index_col=[0], sep='\t', low_memory=False)
bio_matrix = bg_importer.form_bio_matrix(bio_df, GHG)

3. Get characterization factor matrix

In [46]:
def form_cf_matrix2(emission_file: str, method: tuple, emission_list: list) -> np.ndarray:
        """
        Get characterization factor matrix data.

        Parameters:
            * emission_file: The path to the file that needs to be processed. The file includes emission name and emission code column.
            * method: The method used for LCA calculation.
        """
        emission_code = pd.read_csv(emission_file, delimiter=",")
        emission_code = file_preprocessing(emission_file, ",", "exiobase name", emission_list)  # sorting the column order align with the desired order.
        codes = emission_code.iloc[:, -1]
        ipcc = ('ecoinvent-3.11', 'IPCC 2013', 'climate change', 'global warming potential (GWP100)')
        bw_method = bd.Method(ipcc).load()
        method_df = pd.DataFrame(bw_method, columns=["database_code", "cf_number"])
        all_ids = method_df["database_code"].to_list()
        all_codes = [bd.get_node(id = i)['code'] for i in all_ids]
        method_df["code"] =  all_codes
        cf_selected = method_df[method_df["code"].isin(codes)][["code", "cf_number"]]
        cf_dict = cf_selected.set_index("code")["cf_number"].to_dict()
        missing_codes = list(set(codes.unique()) - set(cf_selected["code"]))
        
        cf_matrix = []
        if not missing_codes:
            for code in codes:
                cf_matrix.append(cf_dict.get(code))
        else:
            miss_dict = emission_code[["ecoinvent name", "brightway code"]].set_index("brightway code")["ecoinvent name"].to_dict()
            fixed_codes = []
            for code in missing_codes:
                name = miss_dict.get(code)
                if "Carbon dioxide" in name:
                    cf_dict[code] = 1.0
                    fixed_codes.append(code)
            if missing_codes == fixed_codes:
                for code in codes:
                    cf_matrix.append(cf_dict.get(code))

        cf_matrix = np.diagflat(cf_matrix)

        if len(cf_matrix) != len(emission_code):
            raise ValueError(f"Characterization factor data imcomplete, missing: {missing_codes}")
            
        return cf_matrix
        

        

In [47]:
cf_matrix = form_cf_matrix2(EMISSION_CODE_FILE, METHOD, GHG)

In [48]:
cf_matrix

array([[1.000000e+00, 0.000000e+00, 0.000000e+00, 0.000000e+00,
        0.000000e+00, 0.000000e+00, 0.000000e+00, 0.000000e+00,
        0.000000e+00, 0.000000e+00, 0.000000e+00, 0.000000e+00,
        0.000000e+00, 0.000000e+00, 0.000000e+00, 0.000000e+00,
        0.000000e+00, 0.000000e+00, 0.000000e+00, 0.000000e+00],
       [0.000000e+00, 1.000000e+00, 0.000000e+00, 0.000000e+00,
        0.000000e+00, 0.000000e+00, 0.000000e+00, 0.000000e+00,
        0.000000e+00, 0.000000e+00, 0.000000e+00, 0.000000e+00,
        0.000000e+00, 0.000000e+00, 0.000000e+00, 0.000000e+00,
        0.000000e+00, 0.000000e+00, 0.000000e+00, 0.000000e+00],
       [0.000000e+00, 0.000000e+00, 1.000000e+00, 0.000000e+00,
        0.000000e+00, 0.000000e+00, 0.000000e+00, 0.000000e+00,
        0.000000e+00, 0.000000e+00, 0.000000e+00, 0.000000e+00,
        0.000000e+00, 0.000000e+00, 0.000000e+00, 0.000000e+00,
        0.000000e+00, 0.000000e+00, 0.000000e+00, 0.000000e+00],
       [0.000000e+00, 0.000000e+00, 0

In [49]:
bd.__version__

(4, 3)

In [ ]:
all_codes = []
        for myid in all_ids:
            for i in bd.Database('ecoinvent-3.11-biosphere'):
                if i.id == myid:
                    all_codes.append(i['code'])
        return all_codes

In [25]:
xxx = bd.Database('ecoinvent-3.11-biosphere')

In [32]:
bd.get_node(id = '134929304555262025')['code']

'd2025041-20ee-41bb-b2c8-47631e839ff0'

In [6]:
cf_matrix = bg_importer.form_cf_matrix(EMISSION_CODE_FILE, METHOD, GHG)

ValueError: Columns must be same length as key

4. Build datapackage

In [50]:
dp_builder = DatapackageBuilder()
datapackage_data = dp_builder.prepare_bw_matrix(tech_matrix, bio_matrix, cf_matrix)
datapackage = dp_builder.prepare_datapackage(datapackage_data)

5. Run static simulation

In [51]:
lca_runner = ExiobaseLCA()
lca_runner.perform_static(68, datapackage, EXIOBASE_AGGREGATED_OUTPUT, 0, "static", "RoW-Services")

Brightway calculated lca score: (292263.28661651607, 'RoW-Services')
Static LCA result saved to /Users/massimo/Documents/Git/brightway-exiobase/notebooks/small_output/CASE_0_static_MC_simulations_RoW-Services.csv.


In [62]:
lca_runner = ExiobaseLCA()
lca_runner.perform_static(65, datapackage, EXIOBASE_AGGREGATED_OUTPUT, 0, "static", activities[65])

Brightway calculated lca score: (14155482.51250722, 'RoW-Electricity nec')
Static LCA result saved to /Users/massimo/Documents/Git/brightway-exiobase/notebooks/small_output/CASE_0_static_MC_simulations_RoW-Electricity nec.csv.


In [60]:
#activities = get_bg_activities(EXIOBASE_AGGREGATED_A_FILE, "\t")

df = pd.read_csv(EXIOBASE_AGGREGATED_A_FILE, delimiter= '\t', header=None)
countries = df.iloc[3:, 0].unique().tolist()
sectors = df.iloc[3:, 1].unique().tolist()
activities = [ x + '-' + y for x in countries for y in sectors]

In [61]:
activities[68]

'RoW-Services'

In [74]:
mysector = activities.index('RoW-Steam and hot water supply services')

lca = bc.LCA(
            demand={mysector : 1},
            data_objs=[datapackage],
        )
lca.lci()
lca.lcia()

print(f"Brightway calculated lca score: {lca.score, activities[mysector]}")

Brightway calculated lca score: (53522166.01912047, 'RoW-Steam and hot water supply services')


In [81]:
for i in [1, 3, 75]: 

    lca = bc.LCA(
            demand={i : 1},
            data_objs=[datapackage],
        )
        
    lca.lci()
    lca.lcia()
        
    print(f"Brightway calculated lca score: {lca.score, activities[i]}")

Brightway calculated lca score: (1073470.7980428257, 'EU28-Energy')
Brightway calculated lca score: (326227.80323628965, 'EU28-Industry')
Brightway calculated lca score: (494753.8151243165, 'RoW-Inert/metal/hazardous waste for treatment: landfill')


In [75]:
len(activities)

76

6. Run normal uncertainty simulation

7. Run pedigree uncertainty simulation